1.roughly check the prediction score 

2.find the user unseen movie and sort them! 


In [1]:
import pandas as pd
import numpy as np

In [2]:
rating = pd.read_csv('rating.csv', index_col=0)

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# link = pd.read_csv('link.csv', index_col=0)
# movie = pd.read_csv('movie.csv', index_col=0)
# genome_tags = pd.read_csv('genome_tags.csv', index_col=0)

In [4]:
rating.head()

,movieId,rating,timestamp
userId,,,
1,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
1,32,3.5,2005-04-02 23:33:39
1,47,3.5,2005-04-02 23:32:07
1,50,3.5,2005-04-02 23:29:40


In [5]:
flattened = pd.DataFrame(rating.to_records())

In [6]:
flattened.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
sample = flattened.head(100000)

In [8]:
user_movie_table = pd.pivot_table(sample, index=['userId'], columns=['movieId'])

In [9]:
user_movie_table.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          NaN    3.5    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          NaN    NaN    4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    4.0   
5          NaN    3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ...                                                                   \
movieId  ...   117511 117590 118354 118696 118900 118997 119141 125916 128488   
userId   ...                                                                    
1        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 128594  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  

[5 rows x 8227 columns]

In [10]:
#find two users pearson correlation

In [11]:
def weight_calculator(user_movie_table, index1, index2):
    """find pearson corr between two users
    Parameters
    --------------
    user_movie_table : pd.DataFrame
        user_movie rating info
    index1, index2 : Integer
        pointer in table
    """
    mu = user_movie_table.T
    user1 = mu[index1]
    user2 = mu[index2]
    common = mu[[index1,index2]].dropna()
    user1_watched = user1.dropna()
    user2_watched = user2.dropna()
    
    x1_mean = user1.mean()
    x2_mean = user2.mean()
    numerator = np.array(list(map(lambda x,y : (x-x1_mean)*(x-x2_mean), common[index1],common[index2]))).sum()
    
    left_corner = np.array(list(map(lambda x : (x-x1_mean)**2, user1_watched))).sum()**0.5
    right_corner = np.array(list(map(lambda x : (x-x2_mean)**2, user2_watched))).sum()**0.5
    denominator = left_corner * right_corner
    
    weight_between_two_users = numerator/denominator
    
    return weight_between_two_users

In [12]:
w = weight_calculator(user_movie_table, 1, 2)

In [13]:
w

0.014944306957911126

### => Note! I found the weight for a person to himself is not 1!

# Now, take the user1 as the target. Let's say I want to know whether I should recommend movie1 to him.

user3,6,8,10 has review for movie1

In [81]:
def predict_ur_score(user_movie_table, user_num, movie_index):
    
    mu = user_movie_table.T

    #1.find people who rates movie3
    movie3_rating = user_movie_table.ix[:,movie_index]
    movie3_rating = movie3_rating.dropna() #so I find the movie3 all user ratings
    
    #2.get users who have rated movie3
    movieId = pd.DataFrame(user_movie_table.ix[1,:]).reset_index()['movieId']
    users_movie3 = list(pd.DataFrame(movie3_rating.reset_index())['userId'])
    movie3_rating_score = list(pd.DataFrame(movie3_rating.reset_index())['rating'][movieId[movie_index]])
    
    #3.calculate predict score for user1 of movie3
    numerator_list = []
    denominator_list = []
    for i in range(len(users_movie3)):
        weight_1_i = weight_calculator(user_movie_table, user_num, users_movie3[i])

        useri_watched = mu[users_movie3[i]].dropna()
        xi_mean = useri_watched.mean()

        score_i = movie3_rating_score[i]
        numerator_i = weight_1_i * (score_i - xi_mean)
        numerator_list.append(numerator_i)

        denominator_i = (weight_1_i**2)**0.5
        denominator_list.append(denominator_i)
    
    numerator = np.array(numerator_list).sum()
    denominator = np.array(denominator_list).sum()
    
    user1_watched = mu[user_num].dropna()
    x1_mean = user1_watched.mean()
    socre_1_3 = x1_mean + numerator/denominator
    
    return socre_1_3

In [82]:
socre_1_3 = predict_ur_score(user_movie_table, 1, 2)

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [83]:
socre_1_3

3.371630498700262

# NOW I Want to compare the user1's rating toward movie2 (roughtly)lol

In [17]:
mu = user_movie_table.T
user1 = mu[1]

In [18]:
user1[:10]

        movieId
rating  1          NaN
        2          3.5
        3          NaN
        4          NaN
        5          NaN
        6          NaN
        7          NaN
        8          NaN
        9          NaN
        10         NaN
Name: 1, dtype: float64

In [19]:
score_1_2 = predict_ur_score(user_movie_table, 1, 1)
score_1_2

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


3.386478820108715

#The true value is 3.5 

# NOW, Let's find the user1 unknow movie

In [20]:
def unseen_movie_index(user_num):
    mu = user_movie_table.T
    user1 = mu[user_num]
    
    unseen_index = []
    for i in range(len(user1)):
        if user1[i] >=0:
            pass
        else:
            unseen_index.append(i)
    
    return unseen_index

In [21]:
unseen_index = unseen_movie_index(1)

In [22]:
unseen_index_try = unseen_index[:100]
# unseen_index_try

# NOW, predict all score for those unseen movies for user1

In [25]:
unseen_index_try[29]

32

In [38]:
pd.DataFrame(movie3_rating.reset_index()).head()

,userId,rating
,,34
0,12,4.0
1,13,1.0
2,14,3.0
3,15,4.0
4,24,4.0


In [84]:
movie_pred_score = {}
for i in range(len(unseen_index_try)):
    score_i = predict_ur_score(user_movie_table, 1, unseen_index_try[i])
    movie_pred_score[unseen_index[i]] = score_i
    print(i)

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [85]:
movie_pred_score

{0: 3.961084384339837,
 2: 3.371630498700262,
 3: 3.4674104249175315,
 4: 3.235440738036334,
 5: 4.170381759432949,
 6: 3.2554787499138365,
 7: 2.633624811552021,
 8: 3.2489017953501405,
 9: 3.6312443351766204,
 10: 3.842991294763309,
 11: 2.9835881057486606,
 12: 3.5287175143879113,
 13: 3.259014737203234,
 14: 2.4673605563359993,
 15: 3.980623955750432,
 16: 4.0416910628662945,
 17: 3.6539596972145407,
 18: 2.825131053940107,
 19: 3.1548005136450943,
 20: 3.819698285671535,
 21: 3.5324343076781424,
 22: 3.428677698673857,
 23: 3.525994275458026,
 24: 3.825751486861127,
 25: 3.5417832323101512,
 26: 4.097321095043524,
 27: 4.040182144482476,
 29: 3.3777846158726352,
 30: 3.588449477102516,
 32: 3.4722436213165926,
 33: 3.0852462128829203,
 34: 4.014768328834996,
 35: 4.382032400589102,
 36: 2.1023166023166024,
 37: 3.4127349788699735,
 38: 3.4887582392181318,
 39: 4.1018970398912895,
 40: 3.4774512094455674,
 41: 3.713366883280721,
 42: 2.6575408064653936,
 43: 3.8350133142443434,
 44